In [1]:
import pandas as pd
import numpy as np
import os

In [27]:
# path
path = r"C:\Users\USYS671257\OneDrive - WSP O365\21_31000110.002_Chattanooga TPO Model\model outputs"

# inputs
tripdata  = pd.read_csv(os.path.join(path, "_trip_2.dat"), sep = '\t')
tourdata  = pd.read_csv(os.path.join(path, "_tour_2.dat"), sep = '\t')
perdata = pd.read_csv(os.path.join(path, "_person_2.dat"), sep = '\t')

# parameters
excludeChildren5 = True

In [21]:
def prep_trmode(tripdata):

    tripdata['tripmode'] = 'no mode'
    tripdata.loc[tripdata['mode'] == 1, 'tripmode'] = 'Walk'
    tripdata.loc[tripdata['mode'] == 2, 'tripmode'] = 'Bike'
    tripdata.loc[tripdata['mode'] == 3, 'tripmode'] = 'Drive Alone'
    tripdata.loc[tripdata['mode'] == 4, 'tripmode'] = 'Shared Ride 2'
    tripdata.loc[tripdata['mode'] == 5, 'tripmode'] = 'Shared Ride 3+'
    tripdata.loc[(tripdata['mode'] == 6) & (tripdata['pathtype'] == 3) , 'tripmode'] = 'Transit-local bus'
    tripdata.loc[(tripdata['mode'] == 6) & (tripdata['pathtype'] == 4) , 'tripmode'] = 'Transit-light rail'
    tripdata.loc[(tripdata['mode'] == 6) & (tripdata['pathtype'] == 5) , 'tripmode'] = 'Transit-premium bus'
    tripdata.loc[(tripdata['mode'] == 6) & (tripdata['pathtype'] == 6) , 'tripmode'] = 'Transit-commuter rail'
    tripdata.loc[(tripdata['mode'] == 6) & (tripdata['pathtype'] == 7) , 'tripmode'] = 'Transit-ferry'
    tripdata.loc[tripdata['mode'] == 8, 'tripmode'] = 'School Bus'

    return tripdata



def prep_tomode(tourdata):

    tourdata['tourmode'] = 'no mode'
    tourdata.loc[tourdata['tmodetp'] == 1, 'tourmode'] = 'Walk'
    tourdata.loc[tourdata['tmodetp'] == 2, 'tourmode'] = 'Bike'
    tourdata.loc[tourdata['tmodetp'] == 3, 'tourmode'] = 'Drive Alone'
    tourdata.loc[tourdata['tmodetp'] == 4, 'tourmode'] = 'Shared Ride 2'
    tourdata.loc[tourdata['tmodetp'] == 5, 'tourmode'] = 'Shared Ride 3+'
    tourdata.loc[tourdata['tmodetp'] == 6, 'tourmode'] = 'Walk-Transit'
    tourdata.loc[tourdata['tmodetp'] == 7, 'tourmode'] = 'Drive-Transit'
    tourdata.loc[tourdata['tmodetp'] == 8, 'tourmode'] = 'School Bus'

    return tourdata



def prep_tripdata(tripdata, tourdata, perdata):

    """ prepare tripdata, 
        add tourmode, destination purpose, pptyp from tourdata and perdata  
    """
    
    perdata = perdata[["hhno","pno","pptyp","psexpfac"]]
    tourdata = pd.merge(tourdata, perdata, on =["hhno","pno"], how="left")    
    tourdata["pdpurp2"] = np.where(tourdata.parent==0, tourdata.pdpurp, 8)   # workbased trips

    tourdata = prep_tomode(tourdata)
    tripdata = prep_trmode(tripdata)

    tourdata = tourdata[["hhno","pno","tour","tourmode","pdpurp2","pptyp","psexpfac"]]
    tripdata = pd.merge(tripdata, tourdata, on=["hhno","pno","tour"], how="left")

    if excludeChildren5:
        tripdata = tripdata[tripdata["pptyp"]<8]
        
    return(tripdata)



def summary_trip_mode(dstripdata, purpose):
    
    """ summarize trip mode by tour mode and destination purpose 
        Work 1 
        School 2 
        Escort 3
        Personal_Business 4
        Shop 5
        Meal 6
        SocRec 7
        Workbased 8    
    """
    
    dstripdata = dstripdata[dstripdata["pdpurp2"]==purpose]
    
    column_order = ['Drive Alone', 'Shared Ride 2', 'Shared Ride 3+', 'Drive-Transit', 'Walk-Transit',
                    'Bike','Walk','School Bus']
    index_order = ['Drive Alone', 'Shared Ride 2', 'Shared Ride 3+', 
                    'Transit-local bus', 'Transit-light rail', 'Transit-premium bus','Transit-commuter rail',
                    'Transit-ferry','School Bus','Bike','Walk']
    
    summary = (dstripdata.groupby(["tripmode", "tourmode"])["psexpfac"].
                          sum().
                          reset_index().
                          pivot_table(values='psexpfac', 
                                     index="tripmode",
                                     columns="tourmode",
                                     fill_value=0).
                           reindex(columns=column_order,
                                   index=index_order).
                           fillna(0))
    
    return summary

In [22]:
# clean up trip data
dstripdata = prep_tripdata(tripdata, tourdata, perdata)

In [48]:
# trip mode - Work 1 
summary_trip_mode(dstripdata, purpose=1)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,379663.0,44323.0,18171.0,571.0,12.0,0.0,0.0,0.0
Shared Ride 2,0.0,43963.0,8531.0,48.0,16.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,30107.0,0.0,7.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,470.0,273.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,2.0,9.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# trip mode - School 2 
summary_trip_mode(dstripdata, purpose=2)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,12983.0,2216.0,2423.0,0.0,0.0,0.0,0.0,219.0
Shared Ride 2,0.0,39410.0,21755.0,0.0,11.0,0.0,0.0,5226.0
Shared Ride 3+,0.0,0.0,84690.0,0.0,1.0,0.0,0.0,5477.0
Transit-local bus,0.0,0.0,0.0,0.0,394.0,0.0,0.0,44.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,26.0,0.0,0.0,2.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# trip mode - Escort 3
summary_trip_mode(dstripdata, purpose=3)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,245.0,36569.0,19635.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 2,0.0,61487.0,18212.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,58282.0,0.0,0.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# trip mode - Personal_Business 4
summary_trip_mode(dstripdata, purpose=4)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,97103.0,4991.0,1753.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 2,0.0,58820.0,11212.0,0.0,56.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,48668.0,0.0,16.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,0.0,1154.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# trip mode - Shop 5
summary_trip_mode(dstripdata, purpose=5)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,61710.0,926.0,296.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 2,0.0,39168.0,7216.0,0.0,58.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,29648.0,0.0,17.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,0.0,377.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# trip mode - Meal 6
summary_trip_mode(dstripdata, purpose=6)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,14408.0,1702.0,476.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 2,0.0,32609.0,5069.0,0.0,16.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,21977.0,0.0,5.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,0.0,164.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
# trip mode - SocRec 7
summary_trip_mode(dstripdata, purpose=7)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,44805.0,3096.0,1056.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 2,0.0,39379.0,7518.0,0.0,23.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,35137.0,0.0,7.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,0.0,363.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
# trip mode - Workbased 8
summary_trip_mode(dstripdata, purpose=8)

tourmode,Drive Alone,Shared Ride 2,Shared Ride 3+,Drive-Transit,Walk-Transit,Bike,Walk,School Bus
tripmode,,,,,,,,
Drive Alone,34902.0,1058.0,321.0,0.0,0.0,0.0,0.0,0.0
Shared Ride 2,0.0,8723.0,1378.0,0.0,7.0,0.0,0.0,0.0
Shared Ride 3+,0.0,0.0,5829.0,0.0,3.0,0.0,0.0,0.0
Transit-local bus,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0
Transit-light rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-premium bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-commuter rail,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Transit-ferry,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
School Bus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
